In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
# Installing pre-requisite for saving the model
!pip install pyyaml h5py

In [ ]:
# Mount google drive where dataset is stored at
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import necessary libraries
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.model_selection import train_test_split

import os
import pandas as pd
import tensorflow as tf

In [ ]:
# Configuration settings 
configs = {
    'epochs': 20,
    'batch_size': 64,
    'seed': 7,
    'learning_rate': 1e-3, #0.001
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'optimizer': 'adam',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
    'fc_layer_1_neurons' : 512,
    'fc_layer_2_neurons' : 512,
}

# Initialise Wandb
hassner_evaluation = wandb.init(
    name='hassner_cnn',
    project='CZ4042_Assignment_2',
    config= configs,
)

config = wandb.config
tf.random.set_seed(config.seed)

wandb: Currently logged in as: dloe001 (use `wandb login --relogin` to force relogin)


In [ ]:
# Define the google drive path where data is stored
base_path = '/content/drive/MyDrive/CZ4042 Project Assignment 2/Models/'

# Load preprocessed dataset containing image path and gender as dataframe
df = pd.read_csv(base_path + 'processed_path_and_gender.txt', sep='\t')

# Append the google drive path infront of the image path 
df['data_path'] = base_path + df['data_path'].astype(str)

# Split the dataset 80% train / 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state= config.seed)

In [ ]:
# Initialise ImageDataGenerator 
# Perform rescaling as well as we have no imported pre-processing methods for the images
train_image_generated = ImageDataGenerator(rescale=1./255)
test_image_generated = ImageDataGenerator(rescale=1./255)

# Generate the rescaled train images from the dataframe given 
train_ds = train_image_generated.flow_from_dataframe(
    dataframe = train_df,
    x_col ='data_path',
    y_col = 'gender',
    batch_size = config.batch_size,
    seed = config.seed,
    shuffle = True,
    class_mode ='raw',
    target_size = (224,224),
)

# Generate the rescaled test images from the dataframe given 
test_ds = test_image_generated.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'data_path',
    y_col = 'gender',
    batch_size = config.batch_size,
    seed = config.seed,
    shuffle = True,
    class_mode = 'raw',
    target_size = (224,224),
)

Found 9755 validated image filenames.
Found 2439 validated image filenames.


In [ ]:
# Creating the hassner_cnn model similar to the research paper
def get_hassner_cnn_model():
    
    seq = tf.keras.models.Sequential()

    pool_size = (3, 3)
    strides = (2, 2)
    
    # Input Layer which accepts shape of (224, 244, 3)
    seq.add(tf.keras.layers.InputLayer(input_shape=(224, 224, 3),))    

    seq.add(tf.keras.layers.Conv2D(filters=96, kernel_size=7, activation=config.hidden_activation,))
    
    seq.add(tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides,))
    
    seq.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization,))

    seq.add(tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation=config.hidden_activation,))
    
    seq.add(tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides,))

    seq.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization,))

    seq.add(tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation=config.hidden_activation,))
    
    seq.add(tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=strides,))
    
    seq.add(tf.keras.layers.Flatten())

    seq.add(tf.keras.layers.Dense(units=config.fc_layer_1_neurons, activation=config.hidden_activation,))

    seq.add(tf.keras.layers.Dropout(rate=0.5))

    seq.add(tf.keras.layers.Dense(units=config.fc_layer_2_neurons, activation=config.hidden_activation,))

    seq.add(tf.keras.layers.Dropout(rate=0.5))

    seq.add(tf.keras.layers.Dense(units=1, activation=config.output_activation,))
    
    return seq

In [ ]:
model = get_hassner_cnn_model()

# Compile the model
model.compile(
    optimizer= config.optimizer,
    loss=  config.loss_function,
    metrics= config.metrics,
)

# View summary of model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 218, 218, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 108, 108, 96)      0         
_________________________________________________________________
lambda (Lambda)              (None, 108, 108, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 256)     614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 51, 51, 256)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 51, 51, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 49, 49, 384)       8

In [ ]:
# Checkpoint storing path
checkpoint_path = base_path + "Model Checkpoints/" + "hassner_cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Model checkpoint to store best weights
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath= checkpoint_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Training the model
train_history = model.fit(
    train_ds,
    epochs= config.epochs,
    use_multiprocessing=True,
    callbacks=[model_checkpoint_callback, WandbCallback()],
    validation_data=test_ds,
)

Epoch 1/20
153/153 [==============================] - 2955s 19s/step - loss: 0.7621 - accuracy: 0.5880 - val_loss: 0.6114 - val_accuracy: 0.6950
Epoch 2/20
153/153 [==============================] - 164s 1s/step - loss: 0.5662 - accuracy: 0.7049 - val_loss: 0.4828 - val_accuracy: 0.7634
Epoch 3/20
153/153 [==============================] - 146s 949ms/step - loss: 0.4797 - accuracy: 0.7707 - val_loss: 0.4293 - val_accuracy: 0.7995
Epoch 4/20
153/153 [==============================] - 150s 976ms/step - loss: 0.4079 - accuracy: 0.8144 - val_loss: 0.4085 - val_accuracy: 0.8122
Epoch 5/20
153/153 [==============================] - 145s 944ms/step - loss: 0.3599 - accuracy: 0.8377 - val_loss: 0.3656 - val_accuracy: 0.8307
Epoch 6/20
153/153 [==============================] - 147s 957ms/step - loss: 0.3101 - accuracy: 0.8686 - val_loss: 0.3604 - val_accuracy: 0.8434
Epoch 7/20
153/153 [==============================] - 155s 1s/step - loss: 0.2714 - accuracy: 0.8831 - val_loss: 0.3409 - val_ac

In [ ]:
# Save the hassner model into a .h5 file
model.save(base_path + 'Saved Models and Weights/' + 'my_hassner_model.h5')

In [ ]:
# Evaluates the hassner model and prints its accuracy
loss, acc = model.evaluate(
    test_ds,
    callbacks=[WandbCallback()],
)

print("Val_accuracy", acc)

39/39 [==============================] - 32s 801ms/step - loss: 0.4962 - accuracy: 0.8799
Val_accuracy 0.8798688054084778


In [ ]:
hassner_evaluation.finish()

accuracy,▁▃▄▅▅▆▆▇▇▇▇█████████
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▆▅▅▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▅▆▇▇▇▇▇███▇█▇████
val_loss,█▅▃▃▂▂▁▂▂▃▂▃▄▄▄▅▅▆▆▅
accuracy,0.98186
best_epoch,6
best_val_loss,0.34087
epoch,19
loss,0.05025
val_accuracy,0.87987
